# Conditional Neural Processes (CNP).
[Conditional Neural Processes](https://arxiv.org/pdf/1807.01613.pdf) (CNPs) were
introduced as a continuation of
[Generative Query Networks](https://deepmind.com/blog/neural-scene-representation-and-rendering/)
(GQN) to extend its training regime to tasks beyond scene rendering, e.g. to
regression and classification.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import datetime
import numpy as np
import math
import os
from resolve.utilities import plotting_utils_cnp as plotting
from resolve.utilities import utilities as utils
from resolve.conditional_neural_process import DataGeneration
from resolve.conditional_neural_process import DeterministicModel
from torch.utils.tensorboard import SummaryWriter
import yaml
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score


In [2]:
# Set the path to the yaml settings file here
path_to_settings = "../"
with open(f"{path_to_settings}/settings.yaml", "r") as f:
    config_file = yaml.safe_load(f)

TRAINING_EPOCHS = int(config_file["cnp_settings"]["training_epochs"]) # Total number of training points: training_iterations * batch_size * max_content_points
TEST_AFTER = int(config_file["cnp_settings"]["test_after"])
torch.manual_seed(0)
BATCH_SIZE = config_file["cnp_settings"]["batch_size_train"]
LEARNING_RATE = config_file["cnp_settings"]["learning_rate"]
FILES_PER_BATCH = config_file["cnp_settings"]["files_per_batch_train"]
target_range = config_file["simulation_settings"]["target_range"]
is_binary = target_range[0] >= 0 and target_range[1] <= 1
version = config_file["path_settings"]["version"]
path_out = f'{config_file["path_settings"]["path_out_cnp"]}/{version}'




In [3]:
x_size, y_size = utils.get_feature_and_label_size(config_file)

In [4]:
from tqdm import tqdm
d_x, d_in, representation_size, d_out = x_size , x_size+y_size, config_file["cnp_settings"]["representation_size"], y_size*2
encoder_sizes = [d_in] + config_file["cnp_settings"]["encoder_hidden_layers"] + [representation_size]
decoder_sizes = [representation_size + d_x]+ config_file["cnp_settings"]["decoder_hidden_layers"] + [d_out] 

model = DeterministicModel(encoder_sizes, decoder_sizes)
os.system(f'mkdir -p {path_out}/cnp_{version}_tensorboard_logs/arxiv')
os.system(f'mv {path_out}/cnp_{version}_tensorboard_logs/events* {path_out}/cnp_{version}_tensorboard_logs/arxiv/')
writer = SummaryWriter(log_dir=f'{path_out}/cnp_{version}_tensorboard_logs')

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)


USE_DATA_AUGMENTATION = config_file["cnp_settings"]["use_data_augmentation"]
# load data:
dataset_train = DataGeneration(mode = "training", 
                                config_file=config_file, 
                                path_to_files=config_file["path_settings"]["path_to_files_train"], 
                                use_data_augmentation=USE_DATA_AUGMENTATION, 
                                batch_size=BATCH_SIZE,
                                files_per_batch=FILES_PER_BATCH)
dataset_train.set_loader()
dataloader_train = dataset_train.dataloader

dataset_test = DataGeneration(mode = "training", 
                                config_file=config_file, 
                                path_to_files=config_file["path_settings"]["path_to_files_testing"], 
                                use_data_augmentation=False, 
                                batch_size=BATCH_SIZE,
                                files_per_batch=FILES_PER_BATCH)
dataset_test.set_loader()
dataloader_test = dataset_test.dataloader


bce = nn.BCELoss()

for it_epoch in range(TRAINING_EPOCHS):
    data_iter = iter(dataloader_test)
    for b, batch in tqdm(enumerate(dataloader_train), total=math.ceil(len(dataloader_train)/BATCH_SIZE), desc="Training Epoch {}/{}".format(it_epoch+1, TRAINING_EPOCHS)):
        it_step = it_epoch * len(dataloader_train) + b
        batch_formated=dataset_train.format_batch_for_cnp(batch,config_file["cnp_settings"]["context_is_subset"] )
        # Get the predicted mean and variance at the target points for the testing set
        log_prob, mu, _ = model(batch_formated.query, batch_formated.target_y, is_binary)
        
        # Define the loss
        loss = -log_prob.mean()
        loss.backward()

        # Perform gradient descent to update parameters
        optimizer.step()
    
        # reset gradient to 0 on all parameters
        optimizer.zero_grad()
        
        if is_binary:
            loss_bce = bce(mu, batch_formated.target_y)
        else:
            loss_bce=-1
        
        # Inside your batch loop, right after computing losses:
        writer.add_scalar('Loss/Logprob/train', loss.item(), it_step)
        y_pred = mu[0].detach().cpu().numpy().flatten()
        y_true = batch_formated.target_y[0].detach().cpu().numpy().flatten()
        mae = mean_absolute_error(y_true,y_pred)
        mse = mean_squared_error(y_true,y_pred)
        r2 = r2_score(y_true, y_pred)
        writer.add_scalar('Metric/Mae/Test', mae, it_step)
        writer.add_scalar('Metric/Mse/Test', mse, it_step)
        writer.add_scalar('Metric/R2/Test', r2, it_step)
        if is_binary:
            writer.add_scalar('Loss/BCE/train', loss_bce.item(), it_step)
            y_pred = (y_pred > 0.5).astype(int)
            y_true = y_true.astype(int)
            acc = accuracy_score(y_true, y_pred)
            writer.add_scalar('Accuracy/train', acc, global_step=it_step)
            from sklearn.metrics import precision_score, recall_score, f1_score
            precision = precision_score(y_true, y_pred, zero_division=0)
            recall = recall_score(y_true, y_pred, zero_division=0)
            f1 = f1_score(y_true, y_pred, zero_division=0)
            writer.add_scalar('Metrics/Precision/Train', precision, global_step=it_step)
            writer.add_scalar('Metrics/Recall/Train', recall, global_step=it_step)
            writer.add_scalar('Metrics/F1/Train', f1, global_step=it_step)
        
        mu=mu[0].detach().numpy()
        if b % TEST_AFTER == 0:
            try:
                batch_testing = next(data_iter)
            except StopIteration:
                data_iter = iter(dataloader_test)
                batch_testing = next(data_iter)

            batch_formated_test=dataset_test.format_batch_for_cnp(batch_testing,config_file["cnp_settings"]["context_is_subset"] )
          
            log_prob_testing, mu_testing, _ = model(batch_formated_test.query, batch_formated_test.target_y, is_binary)
            loss_testing = -log_prob_testing.mean()
            
            if is_binary:
                loss_bce_testing = bce(mu_testing,  batch_formated_test.target_y)
            else:
                loss_bce_testing = -1.

            writer.add_scalar('Loss/Logprob/Test', loss_testing.item(), b+it_epoch * len(dataloader_train))
            if is_binary:
                writer.add_scalar('Loss/BCE/Test', loss_bce_testing.item(),b+it_epoch * len(dataloader_train))
                y_pred = (mu_testing[0].detach().cpu().numpy() > 0.5).astype(int).flatten()
                y_true = batch_formated_test.target_y[0].detach().cpu().numpy().astype(int).flatten()
                cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
                fig, ax = plt.subplots()
                sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
                ax.set_xlabel('Predicted Label')
                ax.set_ylabel('True Label')
                ax.set_title(f'Confusion Matrix at step {it_step}')
                writer.add_figure('ConfusionMatrix', fig, global_step=it_step)
                writer.add_scalar('ConfusionMatrix/FalsePositives/Test', cm[0, 1], it_step)
                writer.add_scalar('ConfusionMatrix/FalseNegatives/Test', cm[1, 0], it_step)
                # Predicted probabilities
                acc = accuracy_score(y_true, y_pred)
                writer.add_scalar('Accuracy/Test', acc, global_step=it_step)
                precision = precision_score(y_true, y_pred, zero_division=0)
                recall = recall_score(y_true, y_pred, zero_division=0)
                f1 = f1_score(y_true, y_pred, zero_division=0)
                writer.add_scalar('Metrics/Precision/Train', precision, global_step=it_step)
                writer.add_scalar('Metrics/Recall/Train', recall, global_step=it_step)
                writer.add_scalar('Metrics/F1/Train', f1, global_step=it_step)

            mu_testing = mu_testing[0].detach().numpy()

            y_true = batch_formated_test.target_y[0].detach().numpy()
            mae = mean_absolute_error(y_true,mu_testing)
            mse = mean_squared_error(y_true,mu_testing)
            r2 = r2_score(y_true, y_pred)
            writer.add_scalar('Metric/Mae/Test', mae, it_step)
            writer.add_scalar('Metric/Mse/Test', mse, it_step)
            writer.add_scalar('Metric/R2/Test', r2, it_step)

            if y_size ==1:
                fig = plotting.plot(mu, batch_formated.target_y[0].detach().numpy(), f'{loss:.2f}', mu_testing, batch_formated_test.target_y[0].detach().numpy(), f'{loss_testing:.2f}', target_range, it_step)
                writer.add_figure('Prediction/train_vs_test', fig, global_step=it_step)
            else:
                for k in range(y_size):
                    fig = plotting.plot(mu[:,k], batch_formated.target_y[0].detach().numpy()[:,k], f'{loss:.2f}', mu_testing[:,k], batch_formated_test.target_y[0].detach().numpy()[:,k], f'{loss_testing:.2f}', target_range, it_step)
                    writer.add_figure(f'Prediction/train_vs_test_k{k}', fig, global_step=it_step)



writer.close()
torch.save(model.state_dict(), f'{path_out}/cnp_{version}_model.pth')

config_file["feature_settings"]["x_mean"] = dataset_train.feature_mean.numpy().tolist()
config_file["feature_settings"]["x_std"] = dataset_train.feature_std.numpy().tolist()
# Save back to the file
with open(f"{path_to_settings}/settings.yaml", "w") as f:
    yaml.safe_dump(config_file, f, sort_keys=False)

Data Augmentation in Progress: 100%|██████████| 1000/1000 [00:28<00:00, 35.41it/s]
Training Epoch 1/1: 9948it [24:18, 10.77it/s]                            

Finished full dataset pass. Starting new epoch! 


Training Epoch 1/1: 9950it [24:18,  6.82it/s]


In [5]:
# === Summarize final metrics and hyperparameters ===

# Extract or define final metrics manually if not tracked
final_summary = {
    "Final Hyperparameters": {
        "Encoder Sizes": encoder_sizes,
        "Decoder Sizes": decoder_sizes,
        "Representation Size": representation_size,
        "Learning Rate": LEARNING_RATE,
        "Batch Size": BATCH_SIZE,
        "Files per Batch": FILES_PER_BATCH,
        "Training Epochs": TRAINING_EPOCHS,
        "Binary Task": is_binary,
        "Data Augmentation": config_file["cnp_settings"]["use_data_augmentation"],
        "Context is Subset": config_file["cnp_settings"]["context_is_subset"],
    },
    "Output Path": path_out,
    "Model Checkpoint": f'{path_out}/cnp_{version}_model.pth',
    "TensorBoard Logs": f'{path_out}/cnp_{version}_tensorboard_logs',
}

# Save to readable JSON summary
import json
with open(f"{path_out}/cnp_{version}_summary.json", "w") as f:
    json.dump(final_summary, f, indent=5)

print("Training complete. Summary written to:")
print(f"  - {path_out}/cnp_{version}_model.pth")
print(f"  - {path_out}/cnp_{version}_tensorboard_logs/")
print(f"  - {path_out}/cnp_{version}_summary.json")



Training complete. Summary written to:
  - ../out/cnp/v1.3/cnp_v1.3_model.pth
  - ../out/cnp/v1.3/cnp_v1.3_tensorboard_logs/
  - ../out/cnp/v1.3/cnp_v1.3_summary.json


In [6]:
final_metrics = {
    "Final Train LogProb Loss": loss.item(),
    "Final Train BCE Loss": loss_bce.item() if is_binary else None,
    "Final Test LogProb Loss": loss_testing.item(),
    "Final Test BCE Loss": loss_bce_testing.item() if is_binary else None,
    "Final Accuracy Test": acc,
    "Final Precision": precision,
    "Final Recall": recall,
    "Final F1": f1,
    "Final MAE": mae,
    "Final MSE": mse,
    "Final R2": r2,
}

# Add to summary
final_summary["Final Metrics"] = final_metrics

print(json.dumps(final_summary["Final Metrics"], indent=4))
# Update JSON
with open(f"{path_out}/cnp_{version}_summary.json", "w") as f:
    json.dump(final_summary, f, indent=4)

{
    "Final Train LogProb Loss": 0.6116913557052612,
    "Final Train BCE Loss": 0.3601384162902832,
    "Final Test LogProb Loss": 0.7838737368583679,
    "Final Test BCE Loss": 0.6449772715568542,
    "Final Accuracy Test": 0.52046783625731,
    "Final Precision": 0.15104166666666666,
    "Final Recall": 0.9666666666666667,
    "Final F1": 0.26126126126126126,
    "Final MAE": 0.15875370800495148,
    "Final MSE": 0.056823745369911194,
    "Final R2": 0.7215521335601807
}


#  Start TensorBoard

Run this in terminal:

tensorboard --logdir=\<path to tensor board log dir\> --host=0.0.0.0 --port=7007

Open:

http://localhost:7007/